In [1]:
import sys
sys.path.insert(0, '../../../')

import pandas as pd
from preprocess import Audio_Processor
from sklearn import metrics
from classification_plots import plot_confusion_matrix, plot_learning_curve
import matplotlib.pyplot as plt
import os
import numpy as np
from data_utils import enumerate_strings
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Able to specify which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"]="1"

SR = 16000
blocksize = int(SR/4)
overlap = int(SR/8)

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Setup

In [4]:
path_to_db='../../../../ESC-50/'
ps = Audio_Processor(path_to_db + 'audio/', sr=SR)
dataset = pd.read_csv(path_to_db + 'meta/esc50.csv')
classes = [None] * 50
h_classes = ['Human & Animal', 'Interacting Materials']
mapping = {
    'dog': 0,
    'rooster': 0,
    'pig': 0,
    'cow': 0,
    'frog': 0,
    'cat': 0,
    'hen': 0,
    'insects': 0,
    'sheep': 0,
    'crow': 0,
    'rain': 1,
    'sea_waves': 1,
    'crackling_fire': 1,
    'crickets': 0,
    'chirping_birds': 0,
    'water_drops': 1,
    'wind': 1,
    'pouring_water': 1,
    'toilet_flush': 1,
    'thunderstorm': 1,
    'crying_baby': 0,
    'sneezing': 0,
    'clapping': 1,
    'breathing': 0,
    'coughing': 0,
    'footsteps': 1,
    'laughing': 0,
    'brushing_teeth': 1,
    'snoring': 0,
    'drinking_sipping': 1,
    'door_wood_knock': 1,
    'mouse_click': 1,
    'keyboard_typing': 1,
    'door_wood_creaks': 1,
    'can_opening': 1,
    'washing_machine': 1,
    'vacuum_cleaner': 1,
    'clock_alarm': 1,
    'clock_tick': 1,
    'glass_breaking':1,
    'helicopter': 1,
    'chainsaw': 1,
    'siren': 1,
    'car_horn': 1,
    'engine': 1,
    'train': 1,
    'church_bells': 1,
    'airplane': 1,
    'fireworks': 1,
    'hand_saw': 1,
}
dataset['h_target'] = None
for index, row in dataset.iterrows():
    target = row['target']
    classes[target] = row['category']
    dataset.loc[index, 'h_target'] = mapping[row['category']]

## Getting Preprocessed Data
We allow for previously preprocessed data to be retrieved for faster training turnaround. If the fold has been preprocessed, it is loaded but if not it is processed and saved.

In [ ]:
df = ps.preprocess_fold(dataset, 
                        kind='mfcc', 
                        blocksize=blocksize, 
                        overlap=overlap,
                        folds=10,
                        random_state=42
                       )

In [ ]:
df.describe()

,mfcc_2_std,mfcc_2_mean,mfcc_2_noise,mfcc_3_std,mfcc_3_mean,mfcc_3_noise,mfcc_4_std,mfcc_4_mean,mfcc_4_noise,mfcc_5_std,...,sflat_noise,sroll_std,sroll_mean,sroll_noise,rmse_std,rmse_mean,rmse_noise,h_target,target,fold
count,56099.000000,56099.000000,56099.000000,56099.000000,56099.000000,56099.000000,56099.000000,56099.000000,56099.000000,56099.000000,...,56099.000000,56099.000000,56099.000000,56099.000000,56099.000000,56099.000000,56099.000000,56099.000000,56099.000000,56099.00000
mean,74.620105,48.183455,41.746615,36.657617,34.045151,33.982976,33.437552,33.611680,33.133013,32.483809,...,0.008547,0.013318,13.486833,24.152871,4.923368,22.355371,5.746781,0.677196,25.030678,5.49992
std,31.230411,19.313372,17.422680,14.591616,14.029568,14.178934,13.963747,14.416449,14.385026,13.964810,...,0.526716,0.524775,12.321739,34.662390,7.839330,14.920418,3.465677,0.467553,14.715421,2.87227
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-5.474616,-3.585779,3.528071,6.514861,1.245488,0.000000,0.650231,0.000000,0.000000,1.00000
25%,53.330858,35.571534,29.910516,26.795564,24.166223,23.818980,23.655287,23.459516,23.117232,22.656295,...,-0.359569,-0.354345,9.187240,17.209193,3.313718,14.748890,3.202910,0.000000,12.000000,3.00000
50%,75.005146,47.805901,40.757709,36.499331,34.004899,34.239185,33.723865,33.930289,33.499416,33.001761,...,0.004257,0.011280,11.805980,21.401296,4.415099,18.735426,5.037486,1.000000,25.000000,5.00000
75%,95.725709,60.420961,52.761468,46.419201,43.630307,43.773217,43.025421,43.448220,42.671114,41.954634,...,0.375408,0.377607,15.615003,27.040761,5.641969,24.981150,7.546230,1.000000,39.000000,8.00000
max,245.414157,149.001408,136.748850,104.421258,96.603252,96.343963,126.722028,115.167297,129.519065,112.529035,...,5.035179,4.096196,1506.420443,6785.230410,1532.648277,446.425880,104.800653,1.000000,49.000000,10.00000


In [ ]:
df.head()

,mfcc_2_std,mfcc_2_mean,mfcc_2_noise,mfcc_3_std,mfcc_3_mean,mfcc_3_noise,mfcc_4_std,mfcc_4_mean,mfcc_4_noise,mfcc_5_std,...,sflat_noise,sroll_std,sroll_mean,sroll_noise,rmse_std,rmse_mean,rmse_noise,h_target,target,fold
0,35.667364,14.740245,16.602951,15.005460,16.179240,16.828098,32.462874,45.787937,28.678806,23.643274,...,-0.105465,0.168453,15.864316,28.057951,2.639015,23.832897,2.536619,0,0,4
1,24.805771,51.954547,52.240404,42.053567,46.759439,40.667126,47.195964,37.284998,54.027818,67.440227,...,0.440345,-0.082343,18.026999,16.318312,4.260724,24.808608,7.521734,0,14,3
2,30.063264,42.801611,37.267736,41.166681,46.598448,40.018555,40.541033,32.197760,33.680779,47.689735,...,0.680777,0.348438,26.371719,20.230306,5.590171,26.803396,7.337798,0,14,9
3,41.969316,38.878775,34.944328,35.622214,39.008041,44.226222,27.118159,41.433313,38.097662,48.047401,...,0.045830,0.359896,22.803787,17.894792,6.715089,21.423019,6.900385,0,14,5
4,47.263526,50.109014,37.686563,35.382352,48.070665,44.370691,36.357315,39.031184,55.042561,44.230857,...,0.823898,1.065340,23.950147,17.089338,6.765430,22.916632,8.397942,0,14,1


In [ ]:
from sklearn.model_selection import train_test_split
train, test, = train_test_split(
                        df[df.fold != 1], test_size=0.20, random_state=42)

In [ ]:
train_X = train.drop(['target', 'h_target', 'fold'], axis=1)
train_y = train['h_target']
test_X = test.drop(['target', 'h_target', 'fold'], axis=1)
test_y = test['h_target']

# Training

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_regression
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_X = scaler.fit_transform(train_X)
feat_red = PCA(n_components=30)
feat_red.fit(train_X,y=train_y)
train_X = feat_red.transform(train_X)

params = {
          'kernel': ('linear', 'poly', 'rbf', 'sigmoid'),
          'gamma': ('auto', 'scale'),
          'decision_function_shape': ('ovo', 'ovr')
         }

svm = SVC(probability=True)

clf = GridSearchCV(svm, params, cv=5)

clf.fit(train_X, train_y)

In [ ]:
sorted(clf.cv_results_.keys())

In [ ]:
pd.DataFrame(clf.cv_results_)

In [ ]:
clf.best_estimator_

In [ ]:
test_X = scaler.transform(test_X)
test_X = feat_red.transform(test_X)
pred = clf.predict(test_X)
print(metrics.accuracy_score(test_y, pred))
cm = metrics.confusion_matrix(test_y, pred)
plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, h_classes)
plt.show()

# All Data

In [ ]:
all_df = df[df.fold == 1]

In [ ]:
all_df.head()

In [ ]:
X = all_df.drop(['target', 'h_target', 'fold'], axis=1)
y = all_df['h_target']
X = scaler.transform(X)
X = feat_red.transform(X)

In [ ]:
pred = clf.predict(X)
pred_prob = clf.predict_proba(X)
print(metrics.accuracy_score(y, pred))
cm = metrics.confusion_matrix(y, pred)
plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, h_classes)
plt.show()

In [ ]:
clf.best_estimator_ 

In [ ]:
import seaborn as sns
misclassified = {
    'Actual h_target': [],
    'Actual target': [],
    'Anim_prob': [],
    'Interact_prob': []
}
for row, p, prob, Y in zip(all_df.iterrows(), pred, pred_prob, y):
    Y = int(Y)
    p = int(p)
    if p!=Y:
        misclassified['Actual target'].append(classes[int(row[1]['target'])])
        misclassified['Actual h_target'].append(h_classes[p])
        misclassified['Anim_prob'].append(prob[0])
        misclassified['Interact_prob'].append(prob[1])

misclassified = pd.DataFrame.from_dict(misclassified)
misclassified.head()

In [ ]:
average_prob = misclassified.groupby('Actual h_target').mean()
average_prob.head()

In [ ]:
average_prob = misclassified.groupby('Actual target').mean()
average_prob

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(figsize=(50,10))
sns.countplot(x='Actual target', hue='Actual h_target',data=misclassified)